In [1]:
import sys
sys.path.append("../../nodefeaturing")

## Drug feature extracting

In [ ]:
import importlib
import sys
import json
import pickle
import pandas as pd
import numpy as np
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import generate_entity_embedding
importlib.reload(generate_entity_embedding)
sys.path = original_path
from generate_entity_embedding import generating_pro_feature, generating_drug_feature

In [ ]:
''' drugbankid2smiles '''
file_path = "dti2vec_drugbankid2smiles"
df = pd.read_csv(file_path, header=None, delimiter="\t")  # Assuming the file has no header row
print(len(df))
print(df.iloc[[0,1,300,600,1000]])
df.columns = ["Drug_ID", "Drug"]

''' Load json file for mapping encoding & Drug ID '''
temp = []
json_file = 'node2id.json'
with open(json_file) as f:
    data = json.load(f)

for i_idx, item in enumerate(df["Drug_ID"].copy()):
    for j_idx, d_name in enumerate(data.keys()):
        if d_name == item:
            temp.append(data[d_name])
            break
        elif j_idx +1 == len(data):
            temp.append(np.NaN)

''' get drug features '''
# Select unique rows based on the 'Drug' column
unique_drugs_df = df.drop_duplicates(subset=['Drug']).dropna()
unique_drugs_df = generating_drug_feature(unique_drugs_df)
# print(len(unique_drugs_df))
# print(unique_drugs_df)
tmp = unique_drugs_df[['Drug_ID', 'Drug','MORGAN_Features']]
drug_id_array = tmp['Drug_ID'].values
drug_array = tmp['Drug'].values
drug_feat = tmp['MORGAN_Features']
drug_enco = temp

drug_feat = {'Drug_ID': drug_id_array, 'Drug': drug_array, 'MORGAN_Features': drug_feat, "Drug_enco" : drug_enco}
with open("VEC_drug_feats_MORGAN.pkl", 'wb') as f:
   pickle.dump(drug_feat, f)
print(f"Dictionary saved as pickle file")

In [62]:
import pickle
with open('VEC_drug_feats.pkl', 'rb') as f:
    old = pickle.load(f, encoding='utf-8')
print(old.keys())
print(old['Morgan_Features'].shape)
print(old['Morgan_Features'][0])
print(type(old['Morgan_Features']))
print(type(old['Morgan_Features'][0]))

with open('VEC_drug_feats_MORGAN.pkl', 'rb') as f:
    new = pickle.load(f, encoding='utf-8')
print(new.keys())
print(new['MORGAN_Features'].shape)
print(new['MORGAN_Features'][0])
print(type(new['MORGAN_Features']))
print(type(new['MORGAN_Features'][0]))

dict_keys(['Drug_ID', 'Drug', 'Morgan_Features', 'Drug_enco'])
(1478,)
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'

In [3]:
import pickle
with open('VEC_drug_feats_map4.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x.keys())
print(x)
x['MAP4_Features'] = x.pop('Morgan_Features')
print(x.keys())
print(x)
with open("VEC_drug_feats_map4.pkl", 'wb') as f:
   pickle.dump(x, f)

dict_keys(['Drug_ID', 'Drug', 'Morgan_Features', 'Drug_enco'])
{'Drug_ID': array(['DB00672', 'DB06663', 'DB00586', ..., 'DB00610', 'DB08903',
       'DB00474'], dtype=object), 'Drug': array(['CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl',
       'C1[C@H](CN2[C@@H]1C(=O)N[C@H](C(=O)N[C@@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C2=O)CC3=CC=CC=C3)CC4=CC=C(C=C4)OCC5=CC=CC=C5)CCCCN)CC6=CNC7=CC=CC=C76)C8=CC=CC=C8)OC(=O)NCCN',
       'C1=CC=C(C(=C1)CC(=O)O)NC2=C(C=CC=C2Cl)Cl', ...,
       'C[C@@H]([C@@H](C1=CC(=CC=C1)O)O)N',
       'CN(C)CC[C@@](C1=CC=CC2=CC=CC=C21)([C@H](C3=CC=CC=C3)C4=C(N=C5C=CC(=CC5=C4)Br)OC)O',
       'CCC#CC(C)C1(C(=O)NC(=O)N(C1=O)C)CC=C'], dtype=object), 'Morgan_Features': array([array([0.28844152, 0.03085759, 0.33230764, ..., 0.06655665, 0.05585421,
       0.00247426]),
       array([0.05462653, 0.3620248 , 0.11030577, ..., 0.31672817, 0.16140309,
       0.17364596]),
       array([0.13667219, 0.01604207, 0.00904966, ..., 0.16250673, 0.2598468 ,
       0.20769013]),
       ...,
  

In [4]:
with open('VEC_drug_feats_map4.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x.keys())
print(x)

dict_keys(['Drug_ID', 'Drug', 'Drug_enco', 'MAP4_Features'])
{'Drug_ID': array(['DB00672', 'DB06663', 'DB00586', ..., 'DB00610', 'DB08903',
       'DB00474'], dtype=object), 'Drug': array(['CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl',
       'C1[C@H](CN2[C@@H]1C(=O)N[C@H](C(=O)N[C@@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C2=O)CC3=CC=CC=C3)CC4=CC=C(C=C4)OCC5=CC=CC=C5)CCCCN)CC6=CNC7=CC=CC=C76)C8=CC=CC=C8)OC(=O)NCCN',
       'C1=CC=C(C(=C1)CC(=O)O)NC2=C(C=CC=C2Cl)Cl', ...,
       'C[C@@H]([C@@H](C1=CC(=CC=C1)O)O)N',
       'CN(C)CC[C@@](C1=CC=CC2=CC=CC=C21)([C@H](C3=CC=CC=C3)C4=C(N=C5C=CC(=CC5=C4)Br)OC)O',
       'CCC#CC(C)C1(C(=O)NC(=O)N(C1=O)C)CC=C'], dtype=object), 'Drug_enco': array([ 569., 1030.,  668., ...,  728., 1315.,  240.]), 'MAP4_Features': array([array([0.28844152, 0.03085759, 0.33230764, ..., 0.06655665, 0.05585421,
       0.00247426]),
       array([0.05462653, 0.3620248 , 0.11030577, ..., 0.31672817, 0.16140309,
       0.17364596]),
       array([0.13667219, 0.01604207, 0.00904966,

In [5]:
import pickle
with open('VEC_drug_feats_morgan.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x.keys())
print(x)
x['MORGAN_Features'] = x.pop('Morgan_Features')
print(x.keys())
print(x)
with open("VEC_drug_feats_morgan.pkl", 'wb') as f:
   pickle.dump(x, f)

dict_keys(['Drug_ID', 'Drug', 'Morgan_Features', 'Drug_enco'])
{'Drug_ID': array(['DB00672', 'DB00116', 'DB06663', ..., 'DB00610', 'DB08903',
       'DB00474'], dtype=object), 'Drug': array(['CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl',
       'C1C(NC2=C(N1)N=C(NC2=O)N)CNC3=CC=C(C=C3)C(=O)N[C@@H](CCC(=O)O)C(=O)O',
       'C1[C@H](CN2[C@@H]1C(=O)N[C@H](C(=O)N[C@@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C2=O)CC3=CC=CC=C3)CC4=CC=C(C=C4)OCC5=CC=CC=C5)CCCCN)CC6=CNC7=CC=CC=C76)C8=CC=CC=C8)OC(=O)NCCN',
       ..., 'C[C@@H]([C@@H](C1=CC(=CC=C1)O)O)N',
       'CN(C)CC[C@@](C1=CC=CC2=CC=CC=C21)([C@H](C3=CC=CC=C3)C4=C(N=C5C=CC(=CC5=C4)Br)OC)O',
       'CCC#CC(C)C1(C(=O)NC(=O)N(C1=O)C)CC=C'], dtype=object), 'Morgan_Features': 0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...
4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
6       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [6]:
with open('VEC_drug_feats_morgan.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x.keys())
print(x)

dict_keys(['Drug_ID', 'Drug', 'Drug_enco', 'MORGAN_Features'])
{'Drug_ID': array(['DB00672', 'DB00116', 'DB06663', ..., 'DB00610', 'DB08903',
       'DB00474'], dtype=object), 'Drug': array(['CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl',
       'C1C(NC2=C(N1)N=C(NC2=O)N)CNC3=CC=C(C=C3)C(=O)N[C@@H](CCC(=O)O)C(=O)O',
       'C1[C@H](CN2[C@@H]1C(=O)N[C@H](C(=O)N[C@@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C2=O)CC3=CC=CC=C3)CC4=CC=C(C=C4)OCC5=CC=CC=C5)CCCCN)CC6=CNC7=CC=CC=C76)C8=CC=CC=C8)OC(=O)NCCN',
       ..., 'C[C@@H]([C@@H](C1=CC(=CC=C1)O)O)N',
       'CN(C)CC[C@@](C1=CC=CC2=CC=CC=C21)([C@H](C3=CC=CC=C3)C4=C(N=C5C=CC(=CC5=C4)Br)OC)O',
       'CCC#CC(C)C1(C(=O)NC(=O)N(C1=O)C)CC=C'], dtype=object), 'Drug_enco': [569, nan, 1030, 668, 804, 231, 600, 228, 373, 164, 985, nan, 407, 1480, 234, 175, 166, 1080, nan, 580, 374, 677, 987, 712, 1633, 501, 280, 1049, 281, 482, 272, 1316, 509, 1634, 537, 967, 1078, nan, 270, 558, 443, 347, 779, 165, 468, 680, 823, 258, 1261, 450, 816, 1202, 1112, 460, 641, 532, 

## Target feature extracting

### prot-bert-bfd

In [ ]:
import importlib
import sys
import json
import pickle
import pandas as pd
import numpy as np
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import generate_entity_embedding
importlib.reload(generate_entity_embedding)
sys.path = original_path
from generate_entity_embedding import generating_pro_feature, generating_drug_feature

df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','PROT_BERT_BFD_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['PROT_BERT_BFD_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'PROT_BERT_BFD_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["PROT_BERT_BFD_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["PROT_BERT_BFD_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "PROT_BERT_BFD_Features" : new_x["PROT_BERT_BFD_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_prot_bert_bfd.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")

In [52]:
import random
import pickle
with open('VEC_target_feats_prot_bert_bfd.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(type(x['PROT_BERT_BFD_Features']))
random_integer = random.randint(0, 1384); print(random_integer)
print(x['PROT_BERT_BFD_Features'][random_integer].shape) # (1024, ) 확인
print(type(x['PROT_BERT_BFD_Features'][random_integer]))
print(x)

<class 'pandas.core.series.Series'>
1190
(1024,)
<class 'numpy.ndarray'>
{'Target_ID': array(['Gene::3725', 'Gene::10014', 'Gene::2099', ..., 'Gene::107080644',
       'Gene::', 'Gene::104909134'], dtype=object), 'PROT_BERT_BFD_Features': 0       [0.07999994, 0.07057223, -0.005482291, -0.1017...
1       [0.09861873, 0.03345179, -0.0198907, 0.0481128...
2       [0.0921928, 0.0055588936, 0.045266334, 0.19758...
3       [0.065954186, 0.028363759, 0.0691558, 0.004959...
4       [-0.0011728306, 0.07787305, 0.04118555, 0.1335...
                              ...                        
1380    [0.08340913, 0.024366094, 0.0077468874, 0.0336...
1381    [0.10049024, 0.09608592, -0.09092753, -0.11782...
1382    [0.05332482, 0.0827113, -0.0252028, -0.0764334...
1383    [-0.005920995, -0.071281135, 0.12536465, -0.17...
1384    [0.037600864, 0.09319527, 0.11077085, -0.01084...
Name: PROT_BERT_BFD_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   na

### prostt5

In [ ]:
import importlib
import sys
import json
import pickle
import pandas as pd
import numpy as np
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import generate_entity_embedding
importlib.reload(generate_entity_embedding)
sys.path = original_path
from generate_entity_embedding import generating_pro_feature, generating_drug_feature

df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','PROSTT5_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['PROSTT5_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'PROSTT5_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["PROSTT5_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["PROSTT5_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "PROSTT5_Features" : new_x["PROSTT5_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_prostt5.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")

In [9]:
with open('VEC_target_feats_prostt5.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x['PROSTT5_Features'][1300].shape) # (1024, ) 확인
print(x)

(1024,)
{'Target_ID': array(['Gene::3725', 'Gene::10014', 'Gene::2099', ..., 'Gene::107080644',
       'Gene::', 'Gene::104909134'], dtype=object), 'PROSTT5_Features': 0       [0.29774928, -0.10099035, -0.21374746, -0.0479...
1       [0.27234846, -0.07167395, -0.1250989, -0.00307...
2       [0.08530362, -0.20161305, -0.24574412, -0.0009...
3       [0.17284732, -0.074036844, -0.53658897, -0.079...
4       [0.05752742, -0.18257628, 0.040561356, 0.24134...
                              ...                        
1380    [0.18186754, -0.09916633, -0.32874444, 0.42915...
1381    [0.25534803, 0.14017427, 0.37623963, 0.2106000...
1382    [0.16089754, -0.12235806, 0.090396516, 0.21785...
1383    [0.16730103, -0.2704303, -0.025364595, 0.08650...
1384    [0.15681213, -0.20944996, -0.10326583, -0.2648...
Name: PROSTT5_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]), 'map_arr': array([   0,    1,    2, ..., 1382, 1383, 1384])}


### prot_t5_xl_bfd

In [ ]:
import importlib
import sys
import json
import pickle
import pandas as pd
import numpy as np
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import generate_entity_embedding
importlib.reload(generate_entity_embedding)
sys.path = original_path
from generate_entity_embedding import generating_pro_feature, generating_drug_feature

df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','PROT_T5_XL_BFD_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['PROT_T5_XL_BFD_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'PROT_T5_XL_BFD_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["PROT_T5_XL_BFD_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["PROT_T5_XL_BFD_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "PROT_T5_XL_BFD_Features" : new_x["PROT_T5_XL_BFD_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_prot_t5_xl_bfd.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")

In [8]:
with open('VEC_target_feats_prot_t5_xl_bfd.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
import random
random_integer = random.randint(0, 1384); print(random_integer)
print(x['PROT_T5_XL_BFD_Features'][random_integer].shape) # (1024, ) 확인
print(x)

205
(1024,)
{'Target_ID': array(['Gene::3725', 'Gene::10014', 'Gene::2099', ..., 'Gene::107080644',
       'Gene::', 'Gene::104909134'], dtype=object), 'PROT_T5_XL_BFD_Features': 0       [0.3061483, -0.479783, -0.41335437, 0.57882804...
1       [0.36122397, -0.28566578, 0.2765294, 0.2626283...
2       [0.059911527, -0.3247478, -0.076610416, 0.1895...
3       [0.22034469, -0.08428178, -0.37653744, -0.1772...
4       [-0.15420882, -0.11498539, -0.5919012, -0.2620...
                              ...                        
1380    [0.2510656, 0.036168, -0.11522516, 0.31929836,...
1381    [0.44312462, -0.26340228, 0.25449044, -0.22715...
1382    [0.20614415, -0.0829238, 0.030516902, 0.291382...
1383    [-0.19570357, -0.34753188, 0.06822972, -0.0065...
1384    [0.22904179, -0.19650827, -0.10441386, 0.12113...
Name: PROT_T5_XL_BFD_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]), 'map_arr': array([   0,    1,    2, ..., 1382, 1383, 1

### prot-bert

In [ ]:
import importlib
import sys
import json
import pickle
import pandas as pd
import numpy as np
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import generate_entity_embedding
importlib.reload(generate_entity_embedding)
sys.path = original_path
from generate_entity_embedding import generating_pro_feature, generating_drug_feature

df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','PROT_BERT_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['PROT_BERT_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'PROT_BERT_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["PROT_BERT_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["PROT_BERT_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "PROT_BERT_Features" : new_x["PROT_BERT_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_prot_bert.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")

In [7]:
with open('VEC_target_feats_prot_bert.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
import random
random_integer = random.randint(0, 1384); print(random_integer)
print(x['PROT_BERT_Features'][random_integer].shape) # (1024, ) 확인
print(x)

971
(1024,)
{'Target_ID': array(['Gene::3725', 'Gene::10014', 'Gene::2099', ..., 'Gene::107080644',
       'Gene::', 'Gene::104909134'], dtype=object), 'PROT_BERT_Features': 0       [0.055428114, -0.05638484, -0.14977953, -0.084...
1       [0.1306466, 0.00059073814, -0.12146838, 0.0223...
2       [-0.02885853, 0.119123675, -0.11759645, -0.163...
3       [0.14950863, 0.13119787, -0.16628464, -0.27110...
4       [0.0337875, 0.06081189, 0.02806383, -0.0159795...
                              ...                        
1380    [0.11205415, 0.057612926, -0.08516071, -0.2570...
1381    [0.024919827, 0.085816376, -0.027137723, -0.14...
1382    [0.11519282, 0.051940884, 0.08040943, -0.06652...
1383    [0.08724613, 0.07569128, -0.053993437, -0.0662...
1384    [0.110392936, 0.07712469, -0.05605019, -0.0872...
Name: PROT_BERT_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]), 'map_arr': array([   0,    1,    2, ..., 1382, 1383, 1384])}


### prot_t5_xl_uniref50

In [ ]:
import importlib
import sys
import json
import pickle
import pandas as pd
import numpy as np
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import generate_entity_embedding
importlib.reload(generate_entity_embedding)
sys.path = original_path
from generate_entity_embedding import generating_pro_feature, generating_drug_feature

df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','PROT_T5_XL_UNIREF50_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['PROT_T5_XL_UNIREF50_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'PROT_T5_XL_UNIREF50_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["PROT_T5_XL_UNIREF50_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["PROT_T5_XL_UNIREF50_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "PROT_T5_XL_UNIREF50_Features" : new_x["PROT_T5_XL_UNIREF50_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_prot_t5_xl_uniref50.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")

In [3]:
import random
import pickle
with open('VEC_target_feats_prot_t5_xl_uniref50.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
random_integer = random.randint(0, 1384); print(random_integer)
print(x['PROT_T5_XL_UNIREF50_Features'][random_integer].shape) # (1024, ) 확인
print(x)

1320
(1024,)
{'Target_ID': array(['Gene::3725', 'Gene::10014', 'Gene::2099', ..., 'Gene::107080644',
       'Gene::', 'Gene::104909134'], dtype=object), 'PROT_T5_XL_UNIREF50_Features': 0       [0.23339663, -0.16107665, -0.2483075, 0.454003...
1       [0.19799815, -0.17363514, -0.000113108945, 0.2...
2       [-0.21022151, -0.20351231, -0.03177288, 0.1806...
3       [0.16953245, -0.12390078, 0.11324732, 0.272348...
4       [0.12069299, -0.22523174, -0.17325203, 0.00566...
                              ...                        
1380    [0.1424455, -0.11220649, 0.0985885, 0.03786475...
1381    [0.16387756, -0.43449685, -0.012802006, -0.075...
1382    [0.23513082, -0.1760802, 0.14090289, 0.2739504...
1383    [0.095515124, -0.4050157, -0.016741376, 0.0647...
1384    [0.12722236, -0.07694693, 0.09211067, 0.219167...
Name: PROT_T5_XL_UNIREF50_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]), 'map_arr': array([   0,    1,    2, ..., 13

In [6]:
import random
import pickle
with open('VEC_target_feats_tmp.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
random_integer = random.randint(0, 1384); print(random_integer)
print(x['ProtBERT_Features'][random_integer].shape) # (1024, ) 확인
print(x)

266
(1024,)
{'Target_ID': array(['Gene::1586', 'Gene::91', 'Gene::5291', ..., 'Gene::3782',
       'Gene::8510', 'Gene::6330'], dtype=object), 'ProtBERT_Features': 0       [0.10895072, 0.07753971, 0.14122498, -0.004359...
1       [0.07643488, 0.046613887, 0.11374605, 0.034627...
2       [0.079430476, 0.028320394, 0.066061325, 0.0013...
3       [0.13558188, 0.013845695, 0.12252106, 0.037885...
4       [0.11597116, 0.016289648, 0.10930617, 0.023096...
                              ...                        
1380    [0.026317561, 0.002467662, 0.088176705, 0.0322...
1381    [0.025081605, 0.011302358, 0.13916573, 0.03344...
1382    [0.06859535, -0.012777425, 0.09564611, 0.01458...
1383    [0.05675855, -0.017336804, 0.099945456, 0.0534...
1384    [0.08563626, -0.021550683, 0.13873681, 0.03547...
Name: ProtBERT_Features, Length: 1385, dtype: object, 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]), 'map_arr': array([   0,    1,    2, ..., 1382, 1383, 1384])}


### prot_t5_xxl_uniref50 -> OOM Error

In [ ]:
import importlib
import sys
import pandas as pd
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
import gen_feat_prot_t5_xxl_uniref50
# import generating_feature
# importlib.reload(generating_feature)
importlib.reload(gen_feat_prot_t5_xxl_uniref50)
sys.path = original_path
# from generating_feature import generating_pro_feature, generating_drug_feature
from gen_feat_prot_t5_xxl_uniref50 import generating_pro_feature, generating_drug_feature


df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','ProtBERT_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['ProtBERT_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'ProtBERT_Features': target_feat}


''' change Protein ID (UniProtKB) to Gene Id (NCBI gene (formerly Entrezgene) ID) '''
a = pd.read_csv("convert_table", delimiter= "\t")
new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["ProtBERT_Features"][x_idx])
            break

new_x = {}
new_x["Target_ID"] = new
new_x["ProtBERT_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

''' Convert Gene ID to Encoding number And Build Mapping array for training '''
with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
#print(enco)
new_x["Gene_enco"] = enco
new_x = new_x.sort_values(by = "Gene_enco").reset_index(drop=True)
new_x["mapping_arr"] = range(0,len(new_x))
vec_pfeat = {"Target_ID" : new_x["Target_ID"].values, "ProtBERT_Features" : new_x["ProtBERT_Features"], "Gene_enco" : new_x["Gene_enco"].values, "map_arr" : new_x["mapping_arr"].values}

with open("VEC_target_feats_prot_t5_xxl_uniref50.pkl", 'wb') as f:
    pickle.dump(vec_pfeat, f)
print(f"Dictionary saved as pickle file")

In [1]:
with open('VEC_target_feats_prot_t5_xxl_uniref50.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x)

FileNotFoundError: [Errno 2] No such file or directory: 'VEC_target_feats_prot_t5_xxl_uniref50.pkl'

## github sample

In [ ]:
import torch
from transformers import T5EncoderModel, T5Tokenizer
import re
import numpy as np
import gc

tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_bfd", do_lower_case=False )
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_bfd")

In [ ]:
gc.collect()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model = model.eval()

In [ ]:
sequences_Example = ["A E T C Z A O", "S K T Z P", "S K T Z P Z A O T C"]
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True)
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)
embedding = embedding.last_hidden_state.cpu().numpy()

features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    print(len(attention_mask[seq_num]))
    print(seq_len)
    seq_emd = embedding[seq_num][:seq_len-1]
    features.append(seq_emd)

print(features[0].shape)
print(features[1].shape)
print(features[2].shape)

## [: , 0][0] 확인

In [ ]:
import torch
size = torch.Size([1, 1024, 1024])
seq_tensor = torch.arange(1, size.numel() + 1).reshape(size)
print(seq_tensor.shape)
print(seq_tensor)
print(seq_tensor[:,0].shape)
print(seq_tensor[:,0][0])

### F.Linear 확인

In [1]:
import torch
import torch.nn.functional as F

# Create tensors with the given sizes
input_size = (256, 1024)
weight_size = (16, 1024)
bias_size = (16,)

input_tensor = torch.randn(*input_size)
weight_tensor = torch.randn(*weight_size)
bias_tensor = torch.randn(*bias_size)

print("Input size:", input_tensor.size())
print("Weight size:", weight_tensor.size())
print("Bias size:", bias_tensor.size())
# Check if dimensions match
if input_size[1] == weight_size[1] and weight_size[0] == bias_size[0]:
    output_tensor = F.linear(input_tensor, weight_tensor, bias_tensor)
    print("Linear transformation successful.")
    print("Input size:", input_tensor.size())
    print("Output size:", output_tensor.size())
else:
    print("Dimensions do not match for linear transformation.")


Input size: torch.Size([256, 1024])
Weight size: torch.Size([16, 1024])
Bias size: torch.Size([16])
Linear transformation successful.
Input size: torch.Size([256, 1024])
Output size: torch.Size([256, 16])
